## Generating a System parametrized with a SMIRNOFF force field

This example shows how to generate parameterized `System` object from a SMIRNOFF force field.

Note: This example is adopted from a similar example in the Open Force Field toolkit:  a simulation of a System parametrized with a SMIRNOFF force field. See [here](https://github.com/openforcefield/openforcefield/tree/3de67ca96265a3953c008e8e244c380d4aa81f2e/examples/SMIRNOFF_simulation) or a more recent version of the same example.

In [1]:
from simtk.openmm.app import PDBFile

from openforcefield.typing.engines.smirnoff import ForceField
from openforcefield.topology import Topology, Molecule
from openforcefield.utils import get_data_file_path

from system.system import System
from system.typing.smirnoff import *

# First, grab some reference structures from the toolkit's test suite
pdb_file_path = get_data_file_path('systems/test_systems/1_cyclohexane_1_ethanol.pdb')
pdbfile = PDBFile(pdb_file_path)

ethanol = Molecule.from_smiles('CCO')
cyclohexane = Molecule.from_smiles('C1CCCCC1')

# Create the Open Force Field Topology from an OpenMM Topology object.
omm_topology = pdbfile.topology
off_topology = Topology.from_openmm(omm_topology, unique_molecules=[ethanol, cyclohexane])

# Load the OpenFF "Parsley" force field.
forcefield = ForceField('openff-1.0.0.offxml')

In [2]:
slot_smirks_map = build_slot_smirks_map(forcefield=forcefield, topology=off_topology)

In [3]:
build_smirks_potential_map(forcefield=forcefield, smirks_map=slot_smirks_map)

{'Bonds': {'[#6X4:1]-[#6X4:2]': ParametrizedAnalyticalPotential(name='b1', smirks='[#6X4:1]-[#6X4:2]', expression='1/2*k*(length-length_0)**2', independent_variables={'length_0'}, parameters={'k': <Quantity(531.137373861, 'kilocalorie / angstrom ** 2 / mole')>, 'length': <Quantity(1.520375903275, 'angstrom')>}),
  '[#6:1]-[#8:2]': ParametrizedAnalyticalPotential(name='b14', smirks='[#6:1]-[#8:2]', expression='1/2*k*(length-length_0)**2', independent_variables={'length_0'}, parameters={'k': <Quantity(669.1415170138, 'kilocalorie / angstrom ** 2 / mole')>, 'length': <Quantity(1.414287924481, 'angstrom')>}),
  '[#6X4:1]-[#1:2]': ParametrizedAnalyticalPotential(name='b83', smirks='[#6X4:1]-[#1:2]', expression='1/2*k*(length-length_0)**2', independent_variables={'length_0'}, parameters={'k': <Quantity(758.0931772913, 'kilocalorie / angstrom ** 2 / mole')>, 'length': <Quantity(1.092888378383, 'angstrom')>}),
  '[#8:1]-[#1:2]': ParametrizedAnalyticalPotential(name='b87', smirks='[#8:1]-[#1:2]

In [4]:
#smirks_map = build_smirnoff_map(off_topology, forcefield)

slot_smirks_map['vdW'].values()

dict_values(['[#6X4:1]', '[#6X4:1]', '[#6X4:1]', '[#6X4:1]', '[#6X4:1]', '[#6X4:1]', '[#1:1]-[#6X4]', '[#1:1]-[#6X4]', '[#1:1]-[#6X4]', '[#1:1]-[#6X4]', '[#1:1]-[#6X4]', '[#1:1]-[#6X4]', '[#1:1]-[#6X4]', '[#1:1]-[#6X4]', '[#1:1]-[#6X4]', '[#1:1]-[#6X4]', '[#1:1]-[#6X4]', '[#1:1]-[#6X4]', '[#6X4:1]', '[#6X4:1]', '[#8X2H1+0:1]', '[#1:1]-[#6X4]', '[#1:1]-[#6X4]', '[#1:1]-[#6X4]', '[#1:1]-[#6X4]-[#7,#8,#9,#16,#17,#35]', '[#1:1]-[#6X4]-[#7,#8,#9,#16,#17,#35]', '[#1:1]-[#8]'])

In [5]:
# Parametrize the topology and create an OpenFF System.
system = System(
    topology=off_topology,
    forcefield=forcefield,
    positions=pdbfile.positions,
    box=omm_topology.getPeriodicBoxVectors(),
)

SMIRNOFFHandlerNotImplementedError: SMIRNOFF parameter not implemented here. Tried to parse ParameterHandler of name C


### Run a simulation

We can now use the `System` object to run molecular dynamics trajectories with OpenMM.

In [ ]:
from simtk import openmm, unit

# Propagate the System with Langevin dynamics.
time_step = 2*unit.femtoseconds  # simulation timestep
temperature = 300*unit.kelvin  # simulation temperature
friction = 1/unit.picosecond  # collision rate
integrator = openmm.LangevinIntegrator(temperature, friction, time_step)

# Length of the simulation.
num_steps = 1000  # number of integration steps to run

# Logging options.
trj_freq = 1  # number of steps per written trajectory frame
data_freq = 1  # number of steps per written simulation statistics

# Set up an OpenMM simulation.
simulation = openmm.app.Simulation(omm_topology, system, integrator)

# Set the initial positions.
positions = pdbfile.getPositions() 
simulation.context.setPositions(positions)

# Randomize the velocities from a Boltzmann distribution at a given temperature.
simulation.context.setVelocitiesToTemperature(temperature)

# Configure the information in the output files.
pdb_reporter = openmm.app.PDBReporter('trajectory.pdb', trj_freq)
state_data_reporter = openmm.app.StateDataReporter('data.csv', data_freq, step=True,
                                                   potentialEnergy=True, temperature=True,
                                                   density=True)
simulation.reporters.append(pdb_reporter)
simulation.reporters.append(state_data_reporter)

In [ ]:
import time

print("Starting simulation")
start = time.process_time()

# Run the simulation
simulation.step(num_steps)

end = time.process_time()
print("Elapsed time %.2f seconds" % (end-start))
print("Done!")

If successful, the directory where your jupyter notebook is running should contain a `trajectory.pdb` file that you can visualize and a `data.csv` file including potential energy, density, and temperature of each frame.